In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark= SparkSession \
       .builder \
       .appName("daily_practice") \
       .config('spark.ui.port', '4050') \
       .getOrCreate()


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,869 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [2]:
from google.colab import output
output.serve_kernel_port_as_window(4050, path ='/jobs/index.html')

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# 🚩 **Day 6 - 2025/04/29**

### Q1: Multi-level Aggregation and Prediction
Build a recommendation engine system that analyzes purchase patterns. Your input is a series of user-product interactions with timestamps and you need to find products that are frequently bought together within a 30-day window, then calculate the probability of future purchases based on past behavior.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, count, sum, when, collect_list, explode, array
from pyspark.sql.window import Window

# User purchase history data
purchases = spark.createDataFrame([
    (101, "A123", "2024-01-15", 29.99),
    (101, "B456", "2024-01-18", 15.50),
    (101, "C789", "2024-02-02", 45.00),
    (102, "A123", "2024-01-05", 29.99),
    (102, "D012", "2024-01-20", 18.75),
    (102, "B456", "2024-02-10", 15.50),
    (103, "B456", "2024-02-01", 15.50),
    (103, "C789", "2024-02-05", 45.00),
    (103, "E345", "2024-02-15", 60.25),
    (104, "A123", "2024-01-10", 29.99),
    (104, "C789", "2024-01-25", 45.00),
    (104, "E345", "2024-02-20", 60.25)
], ["user_id", "product_id", "purchase_date", "amount"])
```

Expected output (showing product pairs that are purchased together and their future purchase probability):
```
+----------+----------+--------------------+--------------------+
|product_1 |product_2 |purchase_frequency  |purchase_probability|
+----------+----------+--------------------+--------------------+
|A123      |C789      |2                   |0.667               |
|B456      |C789      |2                   |0.667               |
|A123      |B456      |1                   |0.333               |
...
+----------+----------+--------------------+--------------------+
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, count, sum, when, collect_list, explode, array
from pyspark.sql.window import Window

# User purchase history data
purchases = spark.createDataFrame([
    (101, "A123", "2024-01-15", 29.99),
    (101, "B456", "2024-01-18", 15.50),
    (101, "C789", "2024-02-02", 45.00),
    (102, "A123", "2024-01-05", 29.99),
    (102, "D012", "2024-01-20", 18.75),
    (102, "B456", "2024-02-10", 15.50),
    (103, "B456", "2024-02-01", 15.50),
    (103, "C789", "2024-02-05", 45.00),
    (103, "E345", "2024-02-15", 60.25),
    (104, "A123", "2024-01-10", 29.99),
    (104, "C789", "2024-01-25", 45.00),
    (104, "E345", "2024-02-20", 60.25)
], ["user_id", "product_id", "purchase_date", "amount"])

purchases.printSchema()

purchases.show()

root
 |-- user_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- amount: double (nullable = true)

+-------+----------+-------------+------+
|user_id|product_id|purchase_date|amount|
+-------+----------+-------------+------+
|    101|      A123|   2024-01-15| 29.99|
|    101|      B456|   2024-01-18|  15.5|
|    101|      C789|   2024-02-02|  45.0|
|    102|      A123|   2024-01-05| 29.99|
|    102|      D012|   2024-01-20| 18.75|
|    102|      B456|   2024-02-10|  15.5|
|    103|      B456|   2024-02-01|  15.5|
|    103|      C789|   2024-02-05|  45.0|
|    103|      E345|   2024-02-15| 60.25|
|    104|      A123|   2024-01-10| 29.99|
|    104|      C789|   2024-01-25|  45.0|
|    104|      E345|   2024-02-20| 60.25|
+-------+----------+-------------+------+



In [ ]:
from pyspark.sql.functions import col, abs, datediff, least, greatest


# convert to date
purchases = purchases.withColumn("purchase_date", F.to_date("purchase_date"))

p1 = purchases.alias("p1")
p2 = purchases.alias("p2")

# self join the purchase data for the same user where the difference between purchases is less 30 days and not identical products
product_pairs = p1.join(
    p2,
    on="user_id"
).filter(
    (col("p1.product_id") != col("p2.product_id")) &
    (abs(datediff(col("p1.purchase_date"), col("p2.purchase_date"))) <= 30)
)

# we cannot directly use the p1.product_id and p2.product_id to generate product pairs as there can be duplicate pairs
# A123-B456 is same as B456-A123
# therefore we use least and greatest function which returns the min and max values from a list
product_pairs = product_pairs.withColumn("product_1", least(col("p1.product_id"), col("p2.product_id"))) \
                             .withColumn("product_2", greatest(col("p1.product_id"), col("p2.product_id")))

# for each user shows us only the distinct pair of products
unique_product_pairs = product_pairs.select("user_id", "product_1", "product_2").distinct()

purchase_frequency = unique_product_pairs.groupby(["product_1", "product_2"]).agg(count("user_id").alias("purchase_frequency"))
purchase_frequency.show()

# purchase_probability = frequency / product_1_user_count
product_1_user_count = unique_product_pairs.groupBy("product_1").agg(count("user_id").alias("product_1_user_count"))

+---------+---------+------------------+
|product_1|product_2|purchase_frequency|
+---------+---------+------------------+
|     A123|     C789|                 2|
|     B456|     C789|                 2|
|     A123|     D012|                 1|
|     A123|     B456|                 1|
|     B456|     E345|                 1|
|     C789|     E345|                 2|
|     B456|     D012|                 1|
+---------+---------+------------------+



We're aiming to answer:

> **“Given that a user bought `product_1`, what is the probability that they also bought `product_2` within 30 days?”**

This is a **conditional probability**, written as:
```latex
\[
P(\text{product}_2 \mid \text{product}_1) = \frac{\text{Users who bought both}}{\text{Users who bought product}_1}
\]
```
So:

- The **numerator** is the count of users who bought both products within the 30-day window.
- The **denominator** is the number of users who bought `product_1` at all — which is what you computed in the `product_1_user_count`.

In [ ]:
final_result = purchase_frequency.join(product_1_user_count, on="product_1") \
    .withColumn("purchase_probability", F.round(F.col("purchase_frequency") / F.col("product_1_user_count"), 3)) \
    .select("product_1", "product_2", "purchase_frequency", "purchase_probability")

final_result.show()

+---------+---------+------------------+--------------------+
|product_1|product_2|purchase_frequency|purchase_probability|
+---------+---------+------------------+--------------------+
|     B456|     D012|                 1|                0.25|
|     B456|     E345|                 1|                0.25|
|     B456|     C789|                 2|                 0.5|
|     A123|     B456|                 1|                0.25|
|     A123|     D012|                 1|                0.25|
|     A123|     C789|                 2|                 0.5|
|     C789|     E345|                 2|                 1.0|
+---------+---------+------------------+--------------------+



### Q2: Customer Purchase Segmentation
Create a segmentation analysis for customers based on their purchase behavior. Identify customers who are "high value," "frequent buyers," "occasional buyers," and "one-time purchasers" based on total spending and purchase frequency.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, when, col, datediff, max, min

# Sample customer purchase data
purchases = spark.createDataFrame([
    (101, "A1", "2024-01-15", 120.50),
    (101, "B2", "2024-02-05", 85.75),
    (101, "C3", "2024-03-20", 210.25),
    (102, "A1", "2024-01-20", 45.99),
    (102, "D4", "2024-01-25", 15.25),
    (103, "B2", "2024-02-10", 85.75),
    (103, "E5", "2024-03-05", 150.00),
    (103, "F6", "2024-04-12", 95.50),
    (103, "G7", "2024-04-25", 120.25),
    (104, "H8", "2024-02-28", 35.50),
    (105, "I9", "2024-03-15", 450.75),
    (105, "J10", "2024-04-02", 320.25)
], ["customer_id", "product_id", "purchase_date", "amount"])
```

Expected output:
```
+------------+----------------+------------------+----------------+
|customer_id |total_spent     |purchase_frequency|segment         |
+------------+----------------+------------------+----------------+
|101         |416.50          |3                 |high_value      |
|102         |61.24           |2                 |occasional_buyer|
|103         |451.50          |4                 |frequent_buyer  |
|104         |35.50           |1                 |one_time        |
|105         |771.00          |2                 |high_value      |
+------------+----------------+------------------+----------------+
```

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, when, col, datediff, max, min

# Sample customer purchase data
purchases = spark.createDataFrame([
    (101, "A1", "2024-01-15", 120.50),
    (101, "B2", "2024-02-05", 85.75),
    (101, "C3", "2024-03-20", 210.25),
    (102, "A1", "2024-01-20", 45.99),
    (102, "D4", "2024-01-25", 15.25),
    (103, "B2", "2024-02-10", 85.75),
    (103, "E5", "2024-03-05", 150.00),
    (103, "F6", "2024-04-12", 95.50),
    (103, "G7", "2024-04-25", 120.25),
    (104, "H8", "2024-02-28", 35.50),
    (105, "I9", "2024-03-15", 450.75),
    (105, "J10", "2024-04-02", 320.25)
], ["customer_id", "product_id", "purchase_date", "amount"])

purchases.printSchema()

root
 |-- customer_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- amount: double (nullable = true)



In [ ]:
# change date type
purchases = purchases.withColumn("purchase_date", F.to_date("purchase_date"))

# aggregate per customer
purchases_aggregated = purchases.groupBy("customer_id").agg(
                                                            F.sum(F.col("amount")).alias("total_spent"),
                                                            F.count(F.col("customer_id")).alias("purchase_frequency")
                                                        )

# apply segmentation
purchases_aggregated.withColumn("segment",
                                F.when(F.col("purchase_frequency")==1, "one-time")\
                                .when(F.col("purchase_frequency")<=2, "occasional_buyer")\
                                .when((F.col("total_spent")/F.col("purchase_frequency"))>130, "high-value")\
                                .when(F.col("purchase_frequency")>2, "frequent_buyer")
                                ).orderBy("customer_id").show()

+-----------+-----------+------------------+----------------+
|customer_id|total_spent|purchase_frequency|         segment|
+-----------+-----------+------------------+----------------+
|        101|      416.5|                 3|      high-value|
|        102|      61.24|                 2|occasional_buyer|
|        103|      451.5|                 4|  frequent_buyer|
|        104|       35.5|                 1|        one-time|
|        105|      771.0|                 2|occasional_buyer|
+-----------+-----------+------------------+----------------+



### Q3: Product Affinity Analysis
Analyze which products are frequently purchased together. For each product, find its top 2 most commonly co-purchased products and their co-purchase frequency.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set, explode, size, array, lit, row_number
from pyspark.sql.window import Window

# Sample purchase data with order_id to group items purchased together
orders = spark.createDataFrame([
    (1001, "P1", "2024-01-05"),
    (1001, "P2", "2024-01-05"),
    (1001, "P3", "2024-01-05"),
    (1002, "P2", "2024-01-06"),
    (1002, "P4", "2024-01-06"),
    (1003, "P1", "2024-01-07"),
    (1003, "P3", "2024-01-07"),
    (1004, "P1", "2024-01-08"),
    (1004, "P2", "2024-01-08"),
    (1004, "P5", "2024-01-08"),
    (1005, "P2", "2024-01-09"),
    (1005, "P3", "2024-01-09"),
    (1005, "P4", "2024-01-09"),
    (1006, "P1", "2024-01-10"),
    (1006, "P4", "2024-01-10")
], ["order_id", "product_id", "order_date"])
```

Expected output:
```
+-----------+------------------+---------------+
|product_id |co_purchased_with |co_purchase_count|
+-----------+------------------+---------------+
|P1         |P3                |2              |
|P1         |P2                |2              |
|P2         |P3                |2              |
|P2         |P1                |2              |
|P3         |P1                |2              |
|P3         |P2                |2              |
|P4         |P2                |2              |
|P4         |P3                |1              |
|P5         |P1                |1              |
|P5         |P2                |1              |
+-----------+------------------+---------------+
```

In [ ]:
from pyspark.sql.functions import col, collect_set, explode, lit, row_number, expr
from pyspark.sql.window import Window

# Sample purchase data
orders = spark.createDataFrame([
    (1001, "P1", "2024-01-05"),
    (1001, "P2", "2024-01-05"),
    (1001, "P3", "2024-01-05"),
    (1002, "P2", "2024-01-06"),
    (1002, "P4", "2024-01-06"),
    (1003, "P1", "2024-01-07"),
    (1003, "P3", "2024-01-07"),
    (1004, "P1", "2024-01-08"),
    (1004, "P2", "2024-01-08"),
    (1004, "P5", "2024-01-08"),
    (1005, "P2", "2024-01-09"),
    (1005, "P3", "2024-01-09"),
    (1005, "P4", "2024-01-09"),
    (1006, "P1", "2024-01-10"),
    (1006, "P4", "2024-01-10")
], ["order_id", "product_id", "order_date"])

# Step 1: Get all products per order
order_products = orders.groupBy("order_id") \
    .agg(collect_set("product_id").alias("products"))

order_products.show()

+--------+------------+
|order_id|    products|
+--------+------------+
|    1002|    [P2, P4]|
|    1001|[P2, P1, P3]|
|    1003|    [P1, P3]|
|    1005|[P2, P4, P3]|
|    1004|[P2, P1, P5]|
|    1006|    [P1, P4]|
+--------+------------+



In [ ]:
# Step 2: Explode each product as a base and list all co-products
exploded = order_products.select("order_id", explode("products").alias("product_id"), "products")

exploded.show()

+--------+----------+------------+
|order_id|product_id|    products|
+--------+----------+------------+
|    1002|        P2|    [P2, P4]|
|    1002|        P4|    [P2, P4]|
|    1001|        P2|[P2, P1, P3]|
|    1001|        P1|[P2, P1, P3]|
|    1001|        P3|[P2, P1, P3]|
|    1003|        P1|    [P1, P3]|
|    1003|        P3|    [P1, P3]|
|    1005|        P2|[P2, P4, P3]|
|    1005|        P4|[P2, P4, P3]|
|    1005|        P3|[P2, P4, P3]|
|    1004|        P2|[P2, P1, P5]|
|    1004|        P1|[P2, P1, P5]|
|    1004|        P5|[P2, P1, P5]|
|    1006|        P1|    [P1, P4]|
|    1006|        P4|    [P1, P4]|
+--------+----------+------------+



In [ ]:
# Remove the product_id from the list of products to get only co-products (self exclusion)
exploded = exploded.withColumn("co_products",
                               expr("filter(products, x -> x != product_id)")).drop("products")

exploded.show()

+--------+----------+-----------+
|order_id|product_id|co_products|
+--------+----------+-----------+
|    1002|        P2|       [P4]|
|    1002|        P4|       [P2]|
|    1001|        P2|   [P1, P3]|
|    1001|        P1|   [P2, P3]|
|    1001|        P3|   [P2, P1]|
|    1003|        P1|       [P3]|
|    1003|        P3|       [P1]|
|    1005|        P2|   [P4, P3]|
|    1005|        P4|   [P2, P3]|
|    1005|        P3|   [P2, P4]|
|    1004|        P2|   [P1, P5]|
|    1004|        P1|   [P2, P5]|
|    1004|        P5|   [P2, P1]|
|    1006|        P1|       [P4]|
|    1006|        P4|       [P1]|
+--------+----------+-----------+



In [ ]:
# Step 3: Explode co-products to get pair rows
pairs = exploded.select("product_id", explode("co_products").alias("co_purchased_with"))

pairs.show()

+----------+-----------------+
|product_id|co_purchased_with|
+----------+-----------------+
|        P2|               P4|
|        P4|               P2|
|        P2|               P1|
|        P2|               P3|
|        P1|               P2|
|        P1|               P3|
|        P3|               P2|
|        P3|               P1|
|        P1|               P3|
|        P3|               P1|
|        P2|               P4|
|        P2|               P3|
|        P4|               P2|
|        P4|               P3|
|        P3|               P2|
|        P3|               P4|
|        P2|               P1|
|        P2|               P5|
|        P1|               P2|
|        P1|               P5|
+----------+-----------------+
only showing top 20 rows



In [ ]:
# Step 4: Count the frequency of each pair
pair_counts = pairs.groupBy("product_id", "co_purchased_with").count() \
    .withColumnRenamed("count", "co_purchase_count")

pair_counts.show()

+----------+-----------------+-----------------+
|product_id|co_purchased_with|co_purchase_count|
+----------+-----------------+-----------------+
|        P3|               P2|                2|
|        P1|               P3|                2|
|        P2|               P4|                2|
|        P1|               P4|                1|
|        P1|               P5|                1|
|        P3|               P1|                2|
|        P3|               P4|                1|
|        P5|               P1|                1|
|        P2|               P1|                2|
|        P4|               P3|                1|
|        P4|               P2|                2|
|        P4|               P1|                1|
|        P2|               P5|                1|
|        P2|               P3|                2|
|        P1|               P2|                2|
|        P5|               P2|                1|
+----------+-----------------+-----------------+



In [ ]:
# Step 5: Get top 2 co-purchased products per product
window = Window.partitionBy("product_id").orderBy(col("co_purchase_count").desc(), col("co_purchased_with"))

top2 = pair_counts.withColumn("rank", row_number().over(window)) \
    .filter(col("rank") <= 2).drop("rank")

# Display final result
top2.show(truncate=False)


+----------+-----------------+-----------------+
|product_id|co_purchased_with|co_purchase_count|
+----------+-----------------+-----------------+
|P1        |P2               |2                |
|P1        |P3               |2                |
|P2        |P1               |2                |
|P2        |P3               |2                |
|P3        |P1               |2                |
|P3        |P2               |2                |
|P4        |P2               |2                |
|P4        |P1               |1                |
|P5        |P1               |1                |
|P5        |P2               |1                |
+----------+-----------------+-----------------+



### Q4: Time-based Cohort Analysis
Perform a cohort analysis to understand customer retention over time. Group customers by their first purchase month (cohort) and analyze how many return to make purchases in subsequent months.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, month, year, datediff, min, count, lit, when

# Sample customer purchase data
customer_purchases = spark.createDataFrame([
    (101, "2024-01-15", 120.50),
    (101, "2024-02-05", 85.75),
    (101, "2024-03-20", 210.25),
    (102, "2024-01-20", 45.99),
    (102, "2024-03-15", 65.25),
    (103, "2024-02-10", 85.75),
    (103, "2024-03-05", 150.00),
    (103, "2024-04-12", 95.50),
    (104, "2024-01-28", 35.50),
    (104, "2024-04-02", 120.75),
    (105, "2024-02-15", 450.75),
    (105, "2024-02-22", 320.25),
    (106, "2024-03-10", 75.50),
    (107, "2024-03-18", 95.25),
    (107, "2024-04-05", 35.75)
], ["customer_id", "purchase_date", "amount"])
```

Expected output:
```
+---------------+------------------+------------------+------------------+------------------+
|cohort_month   |month_0_retention |month_1_retention |month_2_retention |month_3_retention |
+---------------+------------------+------------------+------------------+------------------+
|2024-01        |3                 |1                 |1                 |1                 |
|2024-02        |2                 |1                 |1                 |0                 |
|2024-03        |2                 |1                 |0                 |0                 |
|2024-04        |0                 |0                 |0                 |0                 |
+---------------+------------------+------------------+------------------+------------------+
```


In [11]:
from pyspark.sql.functions import col, to_date, month, year, datediff, min, count, lit, when

# Sample customer purchase data
customer_purchases = spark.createDataFrame([
    (101, "2024-01-15", 120.50),
    (101, "2024-02-05", 85.75),
    (101, "2024-03-20", 210.25),
    (102, "2024-01-20", 45.99),
    (102, "2024-03-15", 65.25),
    (103, "2024-02-10", 85.75),
    (103, "2024-03-05", 150.00),
    (103, "2024-04-12", 95.50),
    (104, "2024-01-28", 35.50),
    (104, "2024-04-02", 120.75),
    (105, "2024-02-15", 450.75),
    (105, "2024-02-22", 320.25),
    (106, "2024-03-10", 75.50),
    (107, "2024-03-18", 95.25),
    (107, "2024-04-05", 35.75)
], ["customer_id", "purchase_date", "amount"])

# convert to date type and extract purchase month
customer_purchases = customer_purchases.withColumn("purchase_date", F.to_date("purchase_date"))\
                                        .withColumn("purchase_month", F.date_format("purchase_date", "yyyy-MM"))

# get each customer's first purchase month which will be their cohort
customer_cohorts = customer_purchases.groupBy("customer_id").agg(F.min("purchase_date").alias("first_purchase_date"))\
                  .withColumn("cohort", F.date_format("first_purchase_date", "yyyy-MM"))

# 1. join with purchases table in order to groupby cohort
# 2. calc months between current purchase month and cohort that will be the cohort_index
# 3. group by cohort and cohort_index
customer_purchases.join(customer_cohorts, on=["customer_id"], how="inner")\
                  .withColumn("cohort_index", F.floor(F.months_between(F.col("purchase_date"), F.col("first_purchase_date"))))\
                  .groupBy(["cohort", "cohort_index"]).agg(F.count_distinct("customer_id").alias("customer_count"))\
                  .groupBy("cohort").pivot("cohort_index").agg(F.first("customer_count"))\
                  .show()

+-------+---+----+----+
| cohort|  0|   1|   2|
+-------+---+----+----+
|2024-02|  2|NULL|   1|
|2024-03|  2|NULL|NULL|
|2024-01|  3|   1|   2|
+-------+---+----+----+



*We use `F.first()` which basically picks the number as is since each cohort-cohort_index pair has a single customer count*

### Q5: Advanced Funnel Analysis with Attribution
Analyze a customer journey funnel and attribute conversions to different marketing channels based on customer touchpoints and their timing. Implement both first-touch and last-touch attribution models.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, first, last, when, count, sum, lag, lead
from pyspark.sql.window import Window

# Customer touchpoint data
touchpoints = spark.createDataFrame([
    (1001, "view", "organic_search", "2024-01-05 09:12:33"),
    (1001, "click", "email", "2024-01-05 12:32:14"),
    (1001, "cart_add", "direct", "2024-01-06 15:45:22"),
    (1001, "purchase", "direct", "2024-01-06 16:03:45"),
    (1002, "view", "social_media", "2024-01-07 10:17:52"),
    (1002, "click", "social_media", "2024-01-07 10:19:04"),
    (1002, "cart_add", "retargeting", "2024-01-08 14:22:16"),
    (1002, "view", "email", "2024-01-10 08:47:31"),
    (1002, "cart_add", "email", "2024-01-10 08:52:47"),
    (1002, "purchase", "email", "2024-01-10 09:01:12"),
    (1003, "view", "organic_search", "2024-01-06 11:23:45"),
    (1003, "click", "organic_search", "2024-01-06 11:25:12"),
    (1003, "cart_add", "retargeting", "2024-01-07 16:42:38"),
    (1003, "view", "direct", "2024-01-09 20:14:22"),
    (1004, "view", "paid_search", "2024-01-08 13:12:44"),
    (1004, "click", "paid_search", "2024-01-08 13:14:25"),
    (1004, "cart_add", "paid_search", "2024-01-08 13:17:50"),
    (1004, "purchase", "paid_search", "2024-01-08 13:22:11"),
    (1005, "view", "social_media", "2024-01-05 14:32:19"),
    (1005, "click", "retargeting", "2024-01-06 09:41:53"),
    (1005, "cart_add", "retargeting", "2024-01-06 09:44:27"),
    (1005, "view", "email", "2024-01-07 12:12:36")
], ["user_id", "event", "channel", "timestamp"])

# Purchase value data for attribution
purchases = spark.createDataFrame([
    (1001, "2024-01-06 16:03:45", 125.50),
    (1002, "2024-01-10 09:01:12", 84.75),
    (1004, "2024-01-08 13:22:11", 210.25)
], ["user_id", "purchase_time", "purchase_value"])
```

Expected output:
```
+----------------+----------------+--------------------+---------------------+
|channel         |funnel_views    |first_touch_value   |last_touch_value     |
+----------------+----------------+--------------------+---------------------+
|organic_search  |2               |125.50              |0.00                 |
|social_media    |2               |84.75               |0.00                 |
|paid_search     |1               |210.25              |210.25               |
|email           |2               |0.00                |84.75                |
|direct          |1               |0.00                |125.50               |
|retargeting     |1               |0.00                |0.00                 |
+----------------+----------------+--------------------+---------------------+
```


### Q6: Slowly Changing Dimension Processing
Implement a Type 2 slowly changing dimension (SCD) process to track historical changes in product data. For each product, maintain historical records with effective date ranges when attributes change.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_date, lag, lead, when, row_number
from pyspark.sql.window import Window

# Current product dimension table
current_dim = spark.createDataFrame([
    (101, "Tablet Pro", "Electronics", 499.99, "Active", "2023-12-01", "9999-12-31"),
    (102, "Coffee Maker", "Kitchen", 89.99, "Active", "2023-12-01", "9999-12-31"),
    (103, "Wireless Headphones", "Electronics", 129.99, "Active", "2023-12-15", "9999-12-31"),
    (104, "Fitness Watch", "Wearables", 159.99, "Active", "2024-01-10", "9999-12-31"),
    (105, "Blender", "Kitchen", 79.99, "Active", "2024-01-15", "9999-12-31")
], ["product_id", "product_name", "category", "price", "status", "effective_start", "effective_end"])

# New product data with some changes
new_data = spark.createDataFrame([
    (101, "Tablet Pro", "Electronics", 479.99, "Active"),    # Price changed
    (102, "Coffee Maker", "Kitchen", 89.99, "Discontinued"), # Status changed
    (103, "Wireless Headphones", "Electronics", 129.99, "Active"), # No change
    (104, "Fitness Watch", "Electronics", 159.99, "Active"), # Category changed
    (105, "Blender", "Kitchen", 79.99, "Active"),            # No change
    (106, "Smart Speaker", "Electronics", 89.99, "Active")   # New product
], ["product_id", "product_name", "category", "price", "status"])
```

Expected output:
```
+----------+------------------+------------+-------+------------+----------------+-------------+----------+
|product_id|product_name      |category    |price  |status      |effective_start |effective_end|is_current|
+----------+------------------+------------+-------+------------+----------------+-------------+----------+
|101       |Tablet Pro        |Electronics |499.99 |Active      |2023-12-01      |2024-04-30   |false     |
|101       |Tablet Pro        |Electronics |479.99 |Active      |2024-04-30      |9999-12-31   |true      |
|102       |Coffee Maker      |Kitchen     |89.99  |Active      |2023-12-01      |2024-04-30   |false     |
|102       |Coffee Maker      |Kitchen     |89.99  |Discontinued|2024-04-30      |9999-12-31   |true      |
|103       |Wireless Headphones|Electronics|129.99 |Active      |2023-12-15      |9999-12-31   |true      |
|104       |Fitness Watch     |Wearables   |159.99 |Active      |2024-01-10      |2024-04-30   |false     |
|104       |Fitness Watch     |Electronics |159.99 |Active      |2024-04-30      |9999-12-31   |true      |
|105       |Blender           |Kitchen     |79.99  |Active      |2024-01-15      |9999-12-31   |true      |
|106       |Smart Speaker     |Electronics |89.99  |Active      |2024-04-30      |9999-12-31   |true      |
+----------+------------------+------------+-------+------------+----------------+-------------+----------+
```